### Packages

In [ ]:
## packages
import math
import random
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import train_test_split 
from scipy.stats import linregress
from joblib import Parallel, delayed
import psutil
import re
import torch
from torch import nn
from torch import optim
from torch.utils.data import TensorDataset, DataLoader

### ML Modeling

In [ ]:
## calculate relative PLQY
def rel_PLQY(y_abs, y_area):
    # Convert y_abs and y_area to pytorch tensors if they are numpy arrays
    if not torch.is_tensor(y_abs):
        y_abs = torch.from_numpy(y_abs.astype(np.float32))
    if not torch.is_tensor(y_area):
        y_area = torch.from_numpy(y_area.astype(np.float32))
    # data related to the dye and solvent
    PLQY_ref = 0.1336
    ABS_ref = 0.9115
    area_ref = 198493.1521
    # calculate relative_PLQY
    PLQY_sample = PLQY_ref * (y_area / area_ref) * ((1 - torch.pow(torch.tensor(10), -ABS_ref)) / (1 - torch.pow(torch.tensor(10), -y_abs)))
    return PLQY_sample
## scale output
def non_dim_y(y_initial):
    # Convert y_initial to pytorch tensor if it isn't
    if not torch.is_tensor(y_initial):
        y_initial = torch.from_numpy(y_initial.astype(np.float32))
    # Get min and max info
    min_info, min_idx = torch.min(y_initial, axis=0, keepdim=True)
    max_info, max_idx = torch.max(y_initial, axis=0, keepdim=True)
    statNorm_info = torch.concatenate((min_info, max_info), axis=0)
    # Nondimensionalize the y's
    y_scaled = (y_initial - min_info) / (max_info - min_info)
    return y_scaled, statNorm_info
def dim_y(y_transform, statNorm_info):
    # Get min and max info
    min_info = torch.index_select(statNorm_info, dim=0, index=torch.tensor([0]))
    max_info = torch.index_select(statNorm_info, dim=0, index=torch.tensor([1]))
    # Give y's dimension
    y_scaled = min_info + y_transform * (max_info - min_info)
    return y_scaled
## nondimensionalize inputs
def non_dim_x(x_initial):
    # For doping project
    x_normalized = np.zeros((x_initial.shape[0], x_initial.shape[1] - 3))
    x_normalized[:, 0] = (x_initial[:, 0] - 120) / (180 - 120) # x1 = temperature
    x_normalized[:, 1] = (x_initial[:, 1] - 80) / (120 - 80) # x2 = CuI
    x_normalized[:, 2] = (x_initial[:, 2] - 80) / (120 - 80) # x3 = ZnI2
    x_normalized[:, 3] = (x_initial[:, 3] - 20) / (80 - 20) # x4 =  Cs-Oleate
    x_normalized[:, 4] = (x_initial[:, 4] - 220) / (280 - 220) # x5 = ODE
    return x_normalized
## split dataset to training and validation
def split_set(x, y, train_ratio = 0.8, seed_num = 4862): # important hyper-parameter
    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = train_ratio, random_state = seed_num)
    return x_train, x_test, y_train, y_test
## ENN structure
def structure_info(num_models, layer_min = 8, layer_max = 10, node_min = 20, node_max = 40, seed_num = 100): # should tune these hyperparameters
    random.seed(seed_num)
    detail_all = []
    for i in range (num_models):
        num_layers = random.randint(layer_min, layer_max)
        nodes_list = []
        for j in range(num_layers):
            num_nodes = random.randint(node_min, node_max)
            nodes_list.append(num_nodes)
        detail = [num_layers, nodes_list]
        detail_all.append(detail)
    return detail_all
# Create Neural Network model
class NeuralNetwork(nn.Module):
    def __init__(self, structure, network_type, input_shape, output_shape):
        super(NeuralNetwork, self).__init__() # Initialize the network object using the superclass constructor
        self.layers = nn.ModuleList() #List of layers
        self.network_type = network_type
        input_size = input_shape[-1] # Last dimension stores num of input features
        # If network is a feed-forward neural network
        if self.network_type == "ff":
            for num_layers, nodes_list in structure:
                for num_nodes in nodes_list:
                    self.layers.append(nn.Linear(input_size, num_nodes)) # Apply the linear layer
                    self.layers.append(nn.ReLU()) # Apply the ReLU activation
                    input_size = num_nodes #reset input size to be the number of nodes determined by structure_info function
            self.layers.append(nn.Linear(input_size, output_shape[-1]))  # final layer outputs predicted y's
        # If network is a cascade neural network
        elif self.network_type == "cascade":
            for num_layers, nodes_list in structure:
                for num_nodes in nodes_list:
                    self.layers.append(nn.Linear(input_size, num_nodes)) # Apply the linear layer
                    self.layers.append(nn.ReLU()) # Apply the ReLU activation
                    input_size += num_nodes #reset input size to be the number of nodes determined by structure_info function
            self.output_layer = nn.Linear(input_size, output_shape[-1]) # number of output parameters
    def forward(self, x):
        # If network is a feed-forward neural network
        if self.network_type == "ff": 
            for layer in self.layers:
                x = layer(x)          
            return x
        # If network is a cascade neural network
        elif self.network_type == "cascade":
            outputs = x
            for layer in self.layers:
                if isinstance(layer, nn.Linear):
                    x = layer(outputs)  # Apply the linear layer
                elif isinstance(layer, nn.ReLU):
                    x = layer(x)  # Apply the ReLU activation
                    outputs = torch.cat((outputs, x), dim = outputs.ndim-1) # Concatenate the current output to all previous ReLU outputs, along the last dimension
            x = self.output_layer(outputs)     
            return x
# Early Stopping
class EarlyStopping:
    def __init__(self, patience, min_delta):
        self.patience = patience
        self.min_delta = min_delta
        self.patience_counter = 0
        self.best_model_state = None
        self.min_validation_loss = float('inf')
    def should_stop_early(self, model, current_validation_loss):
        # Reset counter if new minimum validation loss found
        if current_validation_loss < self.min_validation_loss:
            self.min_validation_loss = current_validation_loss
            self.best_model_state = model.state_dict()
            self.patience_counter = 0
        # If validation loss is greater than minimum, increase the patience counter
        elif current_validation_loss > (self.min_validation_loss + self.min_delta):
            self.patience_counter += 1
            if self.patience_counter >= self.patience:
                return True
        return False
    def get_best_model_state(self):
        return self.best_model_state
## Training the neural network using PyTorch
def parallel_train_ML_pytorch(train_x, train_y, val_x, val_y, structure, network_type, batch_size=8, num_epochs=3000, learning_rate=1e-4, patience=500, min_delta=1e-4):
    # Convert dataset to tensors if they are numpy arrays
    if not torch.is_tensor(train_x):
        train_x = torch.from_numpy(train_x.astype(np.float32))
    if not torch.is_tensor(train_y):
        train_y = torch.from_numpy(train_y.astype(np.float32))
    if not torch.is_tensor(val_x):
        val_x = torch.from_numpy(val_x.astype(np.float32))
    if not torch.is_tensor(val_y):
        val_y = torch.from_numpy(val_y.astype(np.float32))
    # Instantiate training and test data
    train_dataset = TensorDataset(train_x, train_y)
    test_dataset = TensorDataset(val_x, val_y)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=len(val_y), shuffle=False)
    # Create model, optimizer, early stopping class, and loss function 
    model = NeuralNetwork(structure=structure, network_type=network_type, input_shape=train_x.shape, output_shape=train_y.shape)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate) #lr - learning step
    early_stopping = EarlyStopping(patience=patience, min_delta=min_delta)
    criterion = nn.MSELoss()
    # Data to store during training
    history = {'epoch': [], 'train_loss': [], 'val_loss': []}
    epochs = []
    train_loss_list = []
    val_loss_list = []
    # Number of outputs to train
    num_outputs = train_y.shape[-1]
    # Train the NN
    for epoch in range(num_epochs):
        ## Model Training
        model.train()
        running_loss = 0.0
        epochs.append(epoch)
        for inputs, labels in train_loader:
            # Reset optimizer
            optimizer.zero_grad()
            # Feed inputs
            outputs = model(inputs)
            # Loss function is the sum of output losses
            loss = None
            for train_output_idx in range(num_outputs):
                train_output = torch.index_select(outputs, dim=outputs.ndim-1, index=torch.tensor(train_output_idx))
                train_label = torch.index_select(labels, dim=labels.ndim-1, index=torch.tensor(train_output_idx))
                if loss is None:
                    loss = criterion(train_output, train_label)
                else:
                    loss += criterion(train_output, train_label)
            # Back propagation
            loss.backward()
            # Update optimizer
            optimizer.step()
            # Add loss to accumulated loss 
            running_loss += loss.item()
        # Compute average loss per batch
        train_loss = running_loss / len(train_loader)
        train_loss_list.append(train_loss)
        ## Model Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            # Validate the NN
            for val_inputs, val_labels in test_loader:
                val_outputs = model(val_inputs)
                val_loss = None
                # Validation loss is the sum of output losses
                for val_output_idx in range(num_outputs):
                    val_output = torch.index_select(val_outputs, dim=val_outputs.ndim-1, index=torch.tensor(val_output_idx))
                    val_label = torch.index_select(val_labels, dim=val_labels.ndim-1, index=torch.tensor(val_output_idx))
                    if val_loss is None:
                        val_loss = criterion(val_output, val_label).item()
                    else:
                        val_loss += criterion(val_output, val_label).item()
            # Compute average loss per batch
            val_loss /= len(test_loader)
            val_loss_list.append(val_loss)
        #print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {epoch_loss:.4f}, Validation Loss: {val_loss:.4f}')
        if early_stopping.should_stop_early(model, val_loss):
            #print(f'Early stopping triggered at epoch {epoch + 1}')
            break
    # Load the best model
    if early_stopping.get_best_model_state() is not None:
        model.load_state_dict(early_stopping.get_best_model_state())
    # Gather history
    history['epoch'] = epochs
    history['train_loss'] = train_loss_list
    history['val_loss'] = val_loss_list
    result = [model, history]
    return result
## deconvolute the model and training history
def deconvolute_ML(ensemble_result):
    ensemble_model = []
    ensemble_history = []
    for element in ensemble_result:
        ensemble_model.append(element[0])
        ensemble_history.append(element[1])
    return ensemble_model, ensemble_history
## plots related to the training process
def learn_plot(history):
    fig = plt.figure()
    # loss values - MSE
    plt.plot(history['train_loss'], color = 'blue')
    plt.plot(history['val_loss'], color = 'red')
    plt.title('Loss (MSE) - Epoch')
    plt.ylabel('Loss (MSE)')
    plt.xlabel('Epoch Number')
    plt.legend(['Training', 'Validation'])
    fig.tight_layout()
    plt.show()
## Plot for predicted vs. true outputs - for individual models
def predict_plot_separate(model, x, y_true):
    if not torch.is_tensor(x):
        x = torch.from_numpy(x.astype(np.float32))
    dim = y_true.shape[-1]
    plt.figure(figsize = (3.5 * dim, 3.5), dpi = 200)
    # Plot each output separately
    for i in range(dim):
        y_true_i = y_true[:,i]
        model_prediction = model(x).detach().numpy()
        y_predicted_i = model_prediction[:, i]
        detail = linregress(y_true_i, y_predicted_i)
        plt.subplot(1, dim, i + 1)
        plt.scatter(y_true_i, y_predicted_i, color = 'blue', alpha = 0.8, label = 'y' + str(i) + ': $R^2 = $' + str(round((detail.rvalue ** 2), 4)))
        plt.plot([0, 1],[0, 1], color = 'red', label = 'Parity')
        # plt.title('y' + str(i + 1))
        plt.ylabel('Predicted')
        plt.xlabel('Actual')
        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        plt.legend() 
    plt.tight_layout()
    plt.show()
## plot related to the predicted vs. true outputs - for the entire ENN 
def predict_plot_mean(ensemble_NN, x, y_true):
    if not torch.is_tensor(x):
        x = torch.from_numpy(x.astype(np.float32))
    y_pred = []
    for model in ensemble_NN:
        y_pred.append(model(x).detach().numpy())
    y_pred_arr = np.concatenate(y_pred, axis = 1)
    dim = y_true.shape[1]
    y_pred_mean = []
    y_pred_std = []
    plt.figure(figsize = (3.5 * dim, 3.5), dpi = 200)
    # Plot each output separately
    for i in range(dim):
        element = y_pred_arr[:, i::dim]
        element_mean = np.mean(element, axis = 1)
        element_std = np.std(element, axis = 1)
        element_detail = linregress(y_true[:, i], element_mean)
        y_pred_mean.append(element_mean[:, np.newaxis])
        y_pred_std.append(element_std[:, np.newaxis])
        plt.subplot(1, dim, i + 1)
        plt.errorbar(y_true[:, i], element_mean, yerr = element_std, fmt = 'o', color = 'blue', alpha = 0.8, markersize = 5, label = 'y' + str(i) + ': $R^2 = $' + str(round((element_detail.rvalue ** 2), 4)))
        plt.plot([0, 1],[0, 1], color = 'red', label = 'Parity')
        # plt.title('y' + str(i + 1))
        plt.ylabel('Predicted')
        plt.xlabel('Actual')
        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        plt.legend()
    plt.tight_layout()
    pred_mean = np.concatenate(y_pred_mean, axis = 1)
    pred_std = np.concatenate(y_pred_std, axis = 1)
    return pred_mean, pred_std
# Predict relative PLQY and compare it to true relative PLQY
def predict_plot_mean_PLQY(ensemble_NN, x, y_true, statNorm_info):
    if not torch.is_tensor(x):
        x = torch.from_numpy(x.astype(np.float32))
    rel_PLQY_pred = []
    for model in ensemble_NN:
        predictions = model(x).detach()
        y_pred_dim = dim_y(predictions, statNorm_info)
        rel_PLQY_pred.append(rel_PLQY(y_pred_dim[:,[0]], y_pred_dim[:,[1]]).numpy())
    rel_PLQY_pred_arr = np.concatenate(rel_PLQY_pred, axis = 1)
    plt.figure(figsize = (3.5, 3.5), dpi = 200)
    rel_PLQY_pred_mean = np.mean(rel_PLQY_pred_arr, axis = 1)
    rel_PLQY_pred_std = np.std(rel_PLQY_pred_arr, axis = 1)
    y_true_dim = dim_y(y_true, statNorm_info)
    rel_PLQY_true = rel_PLQY(y_true_dim[:,0], y_true_dim[:,1]).numpy()
    element_detail = linregress(rel_PLQY_true, rel_PLQY_pred_mean)
    plt.errorbar(rel_PLQY_true, rel_PLQY_pred_mean, yerr = rel_PLQY_pred_std, fmt = 'o', color = 'blue', alpha = 0.8, markersize = 5, label = '$R^2 = $' + str(round((element_detail.rvalue ** 2), 4)))
    plt.plot([0, 1],[0, 1], color = 'red', label = 'Parity')
    plt.title('Relative PLQY')
    plt.ylabel('Predicted')
    plt.xlabel('True')
    plt.xlim([0.0, 0.25])
    plt.ylim([0.0, 0.25])
    plt.legend()
    plt.tight_layout()
    return rel_PLQY_pred_mean, rel_PLQY_pred_std
## save the ENN that's been trained
def save_ENN(ENN, path):
    for n, model in enumerate(ENN):
        model_path = os.path.join(path, str(n))
        os.mkdir(model_path)
        torch.save(model, os.path.join(model_path , "model" + str(n)))
## load the ENN that's been saved
def load_ENN(path):
    model_idx = sorted(os.listdir(path), key = len)
    ensemble = []
    for idx in model_idx:
        model_path = os.path.join(path, idx)
        ensemble.append(torch.load(os.path.join(model_path, "model" + idx)))
    return ensemble

### Important Hyperparameters

In [ ]:
network_type="cascade"
batch_size=16
num_epochs=3000
learning_rate=1e-3
patience=100
min_delta=1e-4
num_models=20

### Dataset / Pre-processing

In [ ]:
path = input("Please enter the path for the directory that includes files: ")
print("------------------------------------------------------------------------------------------------------------")
FR = pd.read_csv(path + "//" + "inputs.csv", header = 0).to_numpy()
y = pd.read_csv(path + "//" + "outputs.csv", header = 0).to_numpy()
x_norm = non_dim_x(FR)
y_norm, statNorm_info = non_dim_y(y)
x_train, x_valid, y_train, y_valid = split_set(x_norm, y_norm)

### Train ENN

In [ ]:
## train the ENN
start_ML = time()
#     p = psutil.Process()
#     p.nice(psutil.REALTIME_PRIORITY_CLASS)
architecture = structure_info(num_models)
result_aggregate_ML = Parallel(n_jobs = -2)(delayed(parallel_train_ML_pytorch)(x_train, y_train, x_valid, y_valid, [architecture[i]], network_type=network_type, batch_size=batch_size, num_epochs=num_epochs, learning_rate=learning_rate, patience=patience, min_delta=min_delta) for i in range(num_models))
ensemble, history = deconvolute_ML(result_aggregate_ML)
end_ML = time()
print("------------------------------------------------------------------------------------------------------------")
print(f"The execution time to train the ensemble NN is {str(end_ML - start_ML)} seconds.")
print("------------------------------------------------------------------------------------------------------------")

### Learning Curve

In [ ]:
# for his in history:
#     learn_plot(his)

### Parity Plots for Individual Models in ENN

In [ ]:
# for model in ensemble:
#     predict_plot_separate(model, x_train, y_train)

In [ ]:
# for model in ensemble:
#     predict_plot_separate(model, x_valid, y_valid)

### Overall ENN Performance on Individual Outputs

In [ ]:
mean_train, std_train = predict_plot_mean(ensemble, x_train, y_train)

In [ ]:
mean_valid, std_valid = predict_plot_mean(ensemble, x_valid, y_valid)

### Overall ENN Performance on PLQY Proxy

In [ ]:
PLQY_pred_mean_train, PLQY_pred_std_train = predict_plot_mean_PLQY(ensemble, x_train, y_train, statNorm_info) 

In [ ]:
PLQY_pred_mean_valid, PLQY_pred_std_valid = predict_plot_mean_PLQY(ensemble, x_valid, y_valid, statNorm_info) 